# IMPORT

In [16]:
# LOAD SYSTEM MODULES
import sys
import json
from operator import itemgetter

# LOAD 3RD PARTY MODULES
import pymongo
import ottype
import psycopg2

# LOAD CUSTOM MODULES
import config

# CONNECT TO DATABASE

In [17]:
def mongodb_connect():
    # CONNECT TO MONGODB
    client = pymongo.MongoClient('mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1' % \
                                 (config.MONGODB['user'],
                                  config.MONGODB['password'],
                                  config.MONGODB['host'],
                                  config.MONGODB['db']), config.MONGODB['port'])
    db = client.elice

    print("=== Collections ===")
    print("\n".join(db.collection_names()))

    return db

def postgres_connect():
    # CONNECT TO POSTGRESQL
    conn = psycopg2.connect("host='%s' dbname='%s' user='%s' password='%s'" % \
                            (config.POSTGRES['host'],
                             config.POSTGRES['db'],
                             config.POSTGRES['user'],
                             config.POSTGRES['password']))
    cur = conn.cursor()

    return cur

In [18]:
elice_mongo = mongodb_connect()
elice_postgres = postgres_connect()

=== Collections ===
exercise_running.outputs
usercode_share.docs
attachments.files
attachments.chunks
usercode_share.ops
attachments.metafiles


# DEFINE GLOBAL VARIABLES

In [19]:
global_user_timestamps = {}
global_user_chars_modified = {}
global_user_files = {}
global_doc_histories = {}

# FETCH LECTURES

In [20]:
def fetch_metadata(elice_postgres, course_id):
    # FETCH LECTURES
    elice_postgres.execute("""SELECT * FROM LECTURE_MODEL
                              WHERE COURSE_ID = %d AND IS_DELETED = False
                              ORDER BY ID""" % \
                           (course_id))
    lectures = []
    for lecture in elice_postgres.fetchall():
        lecture_id = lecture[2]
        lecture_title = lecture[4]
        if 'homework' not in lecture_title.lower(): continue
        print("%d: %s" % (lecture_id, lecture_title))
        lectures.append(lecture)

    # LECTURES AND MATERIAL EXERCISES
    lecture_materials = []

    # FETCH MATERIAL EXERCISES
    for lecture in lectures:
        lecture_id = lecture[2]
        elice_postgres.execute("""SELECT material_exercise_model.id, material_exercise_model.title FROM material_exercise_model
                                  INNER JOIN lecture_page_order_model
                                  ON material_exercise_model.lecture_page_order_id = lecture_page_order_model.id
                                  WHERE lecture_page_order_model.lecture_id = %d
                                  AND material_exercise_model.is_deleted = False
                                  ORDER BY lecture_page_order_model.order_no ASC""" % \
                                  (lecture_id))

        # LIST OF DICTIONARIES
        exercises = [{'id': x[0], 'title': x[1]} for x in elice_postgres.fetchall()]

        # MATERIAL EXERCISES FOR LECTURE_ID
        lecture_materials.append({'id': lecture_id,
                                  'title': lecture[4],
                                  'exercises': exercises})

    print(lecture_materials)
    return lecture_materials

In [21]:
lecture_materials = fetch_metadata(elice_postgres, course_id=config.COURSE['course_id'])

24: Homework 0 (Due 2016-03-15 23:59)
41: Homework 1 (Due 2016-04-01 23:59)
58: Homework 2 (Due: April 14, 23:59)
65: Homework 3 (Due: May 5 23:59)
70: HOMEWORK 4 (DUE: MAY 22 23:59)
80: Homework 5 (DUE: Jun. 7 23:59)
[{'title': 'Homework 0 (Due 2016-03-15 23:59)', 'exercises': [{'title': 'Hello', 'id': 96}, {'title': 'Character Frequency', 'id': 97}, {'title': 'Sum over Even Numbers', 'id': 98}, {'title': 'Reverse the Sentences', 'id': 99}, {'title': 'Number of Islands', 'id': 103}], 'id': 24}, {'title': 'Homework 1 (Due 2016-04-01 23:59)', 'exercises': [{'title': 'Singly Linked List', 'id': 137}, {'title': 'Operations Using Singly Linked List', 'id': 138}, {'title': 'Fibonacci', 'id': 139}, {'title': 'Count Steps', 'id': 141}, {'title': 'Multiple Recursion', 'id': 142}], 'id': 41}, {'title': 'Homework 2 (Due: April 14, 23:59)', 'exercises': [{'title': 'Merge Blocks', 'id': 305}, {'title': 'Stack Calculator (1)', 'id': 306}, {'title': 'Stack Calculator (2)', 'id': 307}, {'title': 'Sta

In [7]:
lecture_materials[2]

{'exercises': [{'id': 305, 'title': 'Merge Blocks'},
  {'id': 306, 'title': 'Stack Calculator (1)'},
  {'id': 307, 'title': 'Stack Calculator (2)'},
  {'id': 308, 'title': 'Stack Calculator (3)'},
  {'id': 309, 'title': 'Breadth First Search (1)'},
  {'id': 310, 'title': 'Breadth First Search (2)'}],
 'id': 58,
 'title': 'Homework 2 (Due: April 14, 23:59)'}

# FETCH DOCUMENT HISTORY

In [8]:
def fetch_docs(elice_mongo, elice_postgres, lecture_id, material_exercise_id):
    # FILTER BY MATERIAL EXERCISE IDS
    usercode_docs = elice_mongo['usercode_share.docs'].find({
                        'material_exercise_id': material_exercise_id
                    })
    
    all_users = set()
    
    for doc in usercode_docs:
        user_id = doc['owner_id']
        all_users.add(user_id)
        
        global_user_files.setdefault(lecture_id, {})
        global_user_files[lecture_id].setdefault(material_exercise_id, {})
        global_user_files[lecture_id][material_exercise_id][user_id] = {}
        fetch_ops(elice_mongo, doc, user_id)
        
    for user_id in all_users:
        user_commits = elice_postgres.execute( \
                         """
                         SELECT T1.created_datetime, T1.input_data, T1.is_commit, T2.auto_score FROM EXERCISE_RUNNING_LOG_MODEL T1
                         INNER JOIN EXERCISE_RUNNING_RESULT_LOG_MODEL T2
                         ON T2.exercise_running_id = T1.id
                         WHERE
                         T1.user_id = %d
                         AND T1.material_exercise_id = %d
                         ORDER BY T1.created_datetime desc
                         """ % (user_id, material_exercise_id))
        user_commits = elice_postgres.fetchall()
        doc_histories = global_doc_histories[user_id]
        
        commit_histories = []
        for user_commit in user_commits:
            commit_histories.append({
                    'type': 'commit',
                    'timestamp': user_commit[0],
                    'instance': user_commit
                })
        for doc in doc_histories:
            commit_histories.append({
                    'type': 'doc',
                    'timestamp': doc['timestamp'],
                    'instance': doc
                })
        
        commit_histories = sorted(commit_histories, key=itemgetter('timestamp'))
        
        current_doc = {}
        last_version = 0
        
        for h in commit_histories:
            inst = h['instance']
            if h['type'] == 'doc':
                current_doc[inst['filename']] = inst['content']
                last_version = inst['version']
            else:
                global_user_files[lecture_id][material_exercise_id][user_id][last_version] = {
                        'files': current_doc,
                        'commit': inst
                    }

In [9]:
def fetch_ops(elice_mongo, doc, user_id): #, user_commits, lecture_id, material_exercise_id, user_id):
    # FILTER BY DOCUMENT OBJECT ID
    ops = elice_mongo['usercode_share.ops'].find({
            'doc_id': doc['_id']
          })
    
    global_doc_histories.setdefault(user_id, [])
    global_doc_histories[user_id].append({
            'doc_id': doc['_id'],
            'version': doc['version'],
            'timestamp': doc['modified_timestamp'],
            'filename': doc['filename'],
            'content': doc['content']
        })
    
    content = doc['content']
    for op in ops:
        striped_op = op['op']
        version = op['version']
        content = ottype.inverse_apply(content, striped_op)
        version_timestamp = op['created_timestamp']

        global_doc_histories[user_id].append({
                'doc_id': doc['_id'],
                'version': op['version'],
                'timestamp': version_timestamp,
                'filename': doc['filename'],
                'content': content
            })

# RUN

In [10]:
global_doc_histories = {}
fetch_docs(elice_mongo, elice_postgres, config.COURSE['lecture_id'], config.COURSE['material_exercise_id'])

# FETCH OPs

In [71]:
_docs = {}

In [72]:
def fetch_docs_get_ops(elice_mongo, elice_postgres, lecture_id, material_exercise_id):
    # FILTER BY MATERIAL EXERCISE IDS
    usercode_docs = elice_mongo['usercode_share.docs'].find({
                        'material_exercise_id': material_exercise_id
                    })
        
    for doc in usercode_docs:    
        user_id = doc['owner_id']
        
        _docs.setdefault(lecture_id, {})
        _docs[lecture_id].setdefault(material_exercise_id, {})
        _docs[lecture_id][material_exercise_id].setdefault(user_id, {})
        _docs[lecture_id][material_exercise_id][user_id].setdefault(doc['filename'], {})
        _docs[lecture_id][material_exercise_id][user_id][doc['filename']]['doc'] = {
            'content': doc['content'],
            'filename': doc['filename'],
            'version': doc['version']
        }
        _docs[lecture_id][material_exercise_id][user_id][doc['filename']]['ops'] = fetch_ops(elice_mongo, doc, user_id)

def fetch_ops(elice_mongo, doc, user_id): #, user_commits, lecture_id, material_exercise_id, user_id):
    # FILTER BY DOCUMENT OBJECT ID
    ops = elice_mongo['usercode_share.ops'].find({
            'doc_id': doc['_id']
          })
    
    ret_ops = []
        
    for op in ops:
        striped_op = op['op']
        version = op['version']
        version_timestamp = op['created_timestamp']

        ret_ops.append({
                'op': op['op'],
                'version': op['version'],
                'timestamp': version_timestamp,
                'filename': doc['filename']
            })
        
    
    return ret_ops

In [73]:
for lecture in lecture_materials:
    for exercise in lecture['exercises']:
        fetch_docs_get_ops(elice_mongo, elice_postgres, lecture['id'], exercise['id'])

In [47]:
lecture_materials

[{'exercises': [{'id': 96, 'title': 'Hello'},
   {'id': 97, 'title': 'Character Frequency'},
   {'id': 98, 'title': 'Sum over Even Numbers'},
   {'id': 99, 'title': 'Reverse the Sentences'},
   {'id': 103, 'title': 'Number of Islands'}],
  'id': 24,
  'title': 'Homework 0 (Due 2016-03-15 23:59)'},
 {'exercises': [{'id': 137, 'title': 'Singly Linked List'},
   {'id': 138, 'title': 'Operations Using Singly Linked List'},
   {'id': 139, 'title': 'Fibonacci'},
   {'id': 141, 'title': 'Count Steps'},
   {'id': 142, 'title': 'Multiple Recursion'}],
  'id': 41,
  'title': 'Homework 1 (Due 2016-04-01 23:59)'},
 {'exercises': [{'id': 305, 'title': 'Merge Blocks'},
   {'id': 306, 'title': 'Stack Calculator (1)'},
   {'id': 307, 'title': 'Stack Calculator (2)'},
   {'id': 308, 'title': 'Stack Calculator (3)'},
   {'id': 309, 'title': 'Breadth First Search (1)'},
   {'id': 310, 'title': 'Breadth First Search (2)'}],
  'id': 58,
  'title': 'Homework 2 (Due: April 14, 23:59)'},
 {'exercises': [{'id'

In [74]:
import json

with open('lecture_materials.json', 'w') as fp:
    json.dump(lecture_materials, fp)

with open('ops.json', 'w') as fp:
    json.dump(_docs, fp)

In [12]:
global_user_files[config.COURSE['lecture_id']][config.COURSE['material_exercise_id']][3]

{0: {'commit': (1459406636620, '', False, None),
  'files': {'cs206c/Main.java': 'package cs206c;\n\nimport cs206c.Block;\nimport java.io.*;\nimport java.util.*;\n\npublic class Main {\n\n\tpublic static void main(String args[]) {\n\t\t// Get the list of blocks\n\t\tList<Block> blocks = readBlocks();\n\n\t\t// Merge the blocks\n\t\tList<Block> mergedBlocks = mergeBlocks(blocks);\n\n\t\t// Print the merged blocks\n\t\tprintBlocks(mergedBlocks);\n\t}\n\n\t// Problem 1\n\t// DO NOT MODIFY THE FUNCTION DECLARATION\n\tpublic static List<Block> readBlocks() {\n\t\t// Implement here\n    List<Block> blocks = new ArrayList<Block>();\n    Scanner in = new Scanner(System.in);\n\n    while (in.hasNextLine()) {\n      String[] splitted = in.nextLine().split(" ");\n      if (splitted.length != 2) {\n        continue;\n      }\n      int start = Integer.parseInt(splitted[0]);\n      int end = Integer.parseInt(splitted[1]);\n      Block block = new Block(start, end);\n      \n      blocks.add(block);

In [15]:
with open("../2016_codeclustering/student_commit_files.json", "w") as fp:
    json.dump(global_user_files, fp)